# 13 Data Validation part 2


The purpose of this notebook is to conduct data validation tests on final_sales.csv. The data validation tests include:

- [ ] Confirm that only tire sales are in the data
 
- [ ] Confirm that the data can be joined correctly to non-sales data (e.g., products.csv., stores.csv, individuals.csv, vehicles.csv)

In [10]:
## TODO - add test complete list so that can store results of each test and print with one command?

## Set up

### Import packages

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import glob
import time
#import geopandas as gpd

#from ydata_profiling import ProfileReport

/Users/jordannieusma/miniconda3/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
pd.options.display.max_columns = 100

### Load final_sales.csv

**final_sales data will be stored in dataframe 'tire_sales'**

In [5]:
# code chunk to load data
time_start = time.time()
# go to directory
%cd /data/p_dsi/teams2023/team7

temp = pd.read_csv('final_sales.csv')
print('Loading complete: final_sales.csv')

tire_sales = temp.drop(['Unnamed: 0'], axis=1)
print('Unnamed column dropped')

print('Parsing complete: tire_sales ready for testing')
print("Info on tire_sales: ", tire_sales.info())

print("Time elapsed: {} seconds".format(time.time()-time_start))

    #################
    ###### TODO #####
    #################
    #fix: run into error when converting NaNs to int
    
    # fix data type of the INDIV_ID col (to int)
    #tire_sales['INDIV_ID'] = tire_sales['INDIV_ID'].astype(int)
    #if tire_sales['INDIV_ID'].dtypes is int:
    #    print('Parsing complete: tire_sales ready for testing')
    #else: print('Parsing error: INDIV_ID column not set to int.')

/gpfs52/data/p_dsi/teams2023/team7
Loading complete: final_sales.csv
Unnamed column dropped
Parsing complete: tire_sales ready for testing
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13499278 entries, 0 to 13499277
Data columns (total 8 columns):
 #   Column      Dtype  
---  ------      -----  
 0   STORE_ID    int64  
 1   TRAN_ID     int64  
 2   DATE        object 
 3   ARTICLE_ID  int64  
 4   INDIV_ID    float64
 5   VEHICLE_ID  int64  
 6   UNITS       float64
 7   SALES       float64
dtypes: float64(3), int64(4), object(1)
memory usage: 823.9+ MB
Info on tire_sales:  None
Time elapsed: 43.599101305007935 seconds


In [6]:
# check dataframe... 
tire_sales.head()

,STORE_ID,TRAN_ID,DATE,ARTICLE_ID,INDIV_ID,VEHICLE_ID,UNITS,SALES
0,244259,992021740,2018-07-22,3431,321025026.0,946172133,1.0,63.99
1,377945,991408880,2018-07-05,3448,321031486.0,968617821,1.0,63.99
2,303595,991638790,2018-07-02,147127,321035434.0,946162627,1.0,64.75
3,239625,991396250,2018-07-02,837,321035850.0,970038529,1.0,73.99
4,16047,991499100,2018-07-19,6172,321024760.0,946194351,2.0,272.98


### Create variables from tire_sales

In [7]:
# get column names
tire_sales_cols = tire_sales.columns.values.tolist()
print("Column names: ", tire_sales_cols)

# get unique values in each col
tire_sales_unique_vals = {}
for col in tire_sales:
    tire_sales_unique_vals.update({col: tire_sales[col].unique()})
    print(col, " added to unique vals dictionary")


Column names:  ['STORE_ID', 'TRAN_ID', 'DATE', 'ARTICLE_ID', 'INDIV_ID', 'VEHICLE_ID', 'UNITS', 'SALES']
STORE_ID  added to unique vals dictionary
TRAN_ID  added to unique vals dictionary
DATE  added to unique vals dictionary
ARTICLE_ID  added to unique vals dictionary
INDIV_ID  added to unique vals dictionary
VEHICLE_ID  added to unique vals dictionary
UNITS  added to unique vals dictionary
SALES  added to unique vals dictionary


## Run tests

### Test 1: Check column names

In [8]:
# navigate to client data folder
%cd /data/p_dsi/teams2023/bridgestone_data/data/

# get col names from all sales csvs
time_start = time.time()
csv_files = glob.glob("/data/p_dsi/teams2023/bridgestone_data/data/*sales_2*.csv")
for df in csv_files:
    temp = pd.read_csv(df, sep = "|", nrows=0)
    if temp.columns.values.tolist() == tire_sales_cols: continue 
    else: print(f"Column does not match in {df}")

print("Parsing complete: all columns match")        
print("Time elapsed: {} seconds".format(time.time()-time_start))

/gpfs52/data/p_dsi/teams2023/bridgestone_data/data
Parsing complete: all columns match
Time elapsed: 37.0685293674469 seconds


### Test 2: Check dates

In [9]:
# create date range to test against
expected_dates = pd.date_range(start='2015-04-01', end='2018-10-31')

tire_sales_dates = np.sort(tire_sales_unique_vals['DATE'])
tire_sales_dates = pd.to_datetime(tire_sales_dates)

if len(expected_dates) - len(tire_sales_dates) != 0:
    unique_values = set(expected_dates) - set(tire_sales_dates)
    print("Difference in date values:", len(expected_dates) - len(tire_sales_dates))
    print("Dates missing from tire_sales:")
    for val in unique_values:
        print(val)
else: print("No dates missing from tire_sales")


Difference in date values: 1
Dates missing from tire_sales:
2017-11-23 00:00:00


### Test 3: Check article_ids

In [22]:
# navigate to client data folder
%cd /data/p_dsi/teams2023/bridgestone_data/data/


# load article ids and category ids from product data
time_start = time.time()
product_ids = pd.read_csv("product.csv", sep="|", usecols=["CATEGORY_CODE", "ARTICLE_ID"])
tire_article_ids = product_ids[product_ids["CATEGORY_CODE"].isin([44.0, 26.0])]
tire_article_ids = set(tire_article_ids['ARTICLE_ID'])
                                                              
# get also non-tire ids
non_tire_article_ids = product_ids[~product_ids["CATEGORY_CODE"].isin([44.0, 26.0])]
non_tire_article_ids = set(non_tire_article_ids['ARTICLE_ID'])

# find intersection between list of tire_article_ids and non_tire_article_ids
check_tire_ids = tire_article_ids.intersection(non_tire_article_ids)

# print results of validation test
if check_tire_ids:
    print("FAIL: Values in tire_article_ids and non_tire_article_ids have matches")
else:
    print("PASS: Values in tire_article_ids and non_tire_article_ids do not match")
      
print("Time elapsed: {} seconds".format(time.time()-time_start))

/gpfs52/data/p_dsi/teams2023/bridgestone_data/data
PASS: Values in tire_article_ids and non_tire_article_ids do not match
Time elapsed: 2.0197794437408447 seconds


### Test 4: Check for missing states and zip codes

In [12]:
# load stores dataset
stores = pd.read_csv("store.csv", sep="|", usecols=["STORE_ID", "STATE_CODE", "ZIP_CODE"])
all_zips = set(stores["ZIP_CODE"])

# join stores and final_sales
tire_sales_stores = tire_sales.merge(stores, on='STORE_ID')

if len(set(tire_sales_stores['STATE_CODE'])) == 48:
    print("PASS: tire_sales contains expected number of state codes (48)")
    
tire_sales_zips = set(tire_sales_stores["ZIP_CODE"])

# find intersection between zip codes in products and tire_sales
check_zips = tire_sales_zips.intersection(all_zips)
if len(check_zips) == len(all_zips):
    print("RESULTS: All zip codes represented in tire_sales")
else: 
    print("RESULTS: Some zip codes are not represented in tire_sales.")

PASS: tire_sales contains expected number of state codes (48)
RESULTS: Some zip codes are not represented in tire_sales.


In [13]:
print("Total possible zip codes: ", len(all_zips))
print("\nZip codes in tire_sales: ", len(check_zips)) 
print("\nDifference: ", len(all_zips)-len(check_zips))

Total possible zip codes:  2374

Zip codes in tire_sales:  2021

Difference:  353


In [14]:
# take a look at data table filtered by missing zipcodes
a = stores[stores["ZIP_CODE"].isin(check_zips)]

# get a df from tire_sales to join with stores
b = tire_sales[["STORE_ID","DATE"]]
missing_zips_df = a.merge(b, on='STORE_ID')

print("No. of states with 'missing' zip codes:", len(set(missing_zips_df["STATE_CODE"])))
print("No. of stores with 'missing' zip codes:", len(set(missing_zips_df["STORE_ID"])))
print("No. of dates with 'missing' zip codes:", len(set(missing_zips_df["DATE"])))

No. of states with 'missing' zip codes: 48
No. of stores with 'missing' zip codes: 2310
No. of dates with 'missing' zip codes: 1309


### Test 5: Check join onto all datasets

In [15]:
#%reset_selective temp

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [ ]:
## TODO - START HERE

In [16]:
# navigate to client data folder
%cd /data/p_dsi/teams2023/bridgestone_data/data/

# load data from individual, product, store, and vehicle csvs
temp = pd.read_csv("product.csv", sep="|")
products = temp[temp["CATEGORY_CODE"].isin([44.0, 26.0])]
print("Data loaded: products")

stores = pd.read_csv("store.csv", sep="|", usecols=["STORE_ID", "STATE_CODE", "ZIP_CODE"])
print("Data loaded: stores")

indivs = pd.read_csv("individual.csv")
print("Data loaded: individuals")

vehicles = pd.read_csv("vehicle.csv", sep="|")
print("Data loaded: vehicles")

/gpfs52/data/p_dsi/teams2023/bridgestone_data/data
Data loaded: products
Data loaded: stores
Data loaded: individuals
Data loaded: vehicles


In [22]:
# join data to tire_sales
tires_prods = tire_sales.merge(products, on="ARTICLE_ID")
tires_prods_stores = tires_prods.merge(stores, on="STORE_ID")
tires_prods_stores_indivs = tires_prods_stores.merge(indivs, on="INDIV_ID")
tire_sales_merged = tires_prods_stores_indivs.merge(vehicles, on="VEHICLE_ID")

KeyError: 'INDIV_ID'

In [ ]:
tire_sales_merged.head()

### Test 6: Check for duplicate transactions

Motivation: Transactions (represented as the TRAN_ID values) should be unique to a date and store. The observations in the dataset are expected to be unique across all the columns in the datasets (STORE_ID, TRAN_ID, DATE, ARTICLE_ID, INDIV_ID, VEHICLE_ID, UNITS, SALES). A validation test should identify any duplicated observations and the quantity of duplicates. 

In [3]:
#%cd ..

#### Test case: February 2018 sales data

In [44]:
%%time

duplicate_test = pd.read_csv("sales_20180228.csv", sep="|", parse_dates=["DATE"],
                             usecols=["STORE_ID","TRAN_ID","ARTICLE_ID","INDIV_ID","VEHICLE_ID","DATE","UNITS","SALES"], 
                             dtype={"STORE_ID":np.int64,
                                    "TRAN_ID":np.int64,
                                    "ARTICLE_ID":np.int64,
                                    #"INDIV_ID":np.int64, # cannot safely be cast?
                                    "VEHICLE_ID":int,
                                    "UNITS":np.float64,
                                    "SALES":np.float64},
                             low_memory=False)
print("Parsing complete. Shape of dataframe: ", duplicate_test.shape)

Parsing complete. Shape of dataframe:  (13247533, 8)
CPU times: user 4.12 s, sys: 1.04 s, total: 5.16 s
Wall time: 5.28 s


In [26]:
# Group by all columns and count the number of duplicates
duplicates = duplicate_test.groupby(duplicate_test.columns.tolist()).size().reset_index().rename(columns={0:'count'})

# Filter the dataframe to keep only rows that have more than one occurrence
duplicates = duplicates[duplicates['count'] > 1]

##### Percent rows duplicated

In [102]:
print("Percent of rows that are duplicated: ", duplicates["count"].sum()/duplicate_test.shape[0]*100)

Percent of rows that are duplicated:  17.606395092580634


In [30]:
# take a look at the highest count values
duplicates.sort_values("count", ascending=False).head(10)

,STORE_ID,TRAN_ID,DATE,ARTICLE_ID,INDIV_ID,VEHICLE_ID,UNITS,SALES,count
11649961,777320,990154510,2018-02-24,7001674,490558891.0,964296191,0.0,0.00,333
11649978,777320,990154510,2018-02-24,7046930,490558891.0,964296191,0.0,0.00,111
11649176,777320,990153760,2018-02-22,7074810,490558891.0,965993351,0.0,0.00,111
11646589,777320,990150440,2018-02-10,7009500,490558891.0,967184333,0.0,79.99,111
11646591,777320,990150440,2018-02-10,7046930,490558891.0,967184333,0.0,0.00,111
11647269,777320,990151510,2018-02-14,7004584,490558891.0,966887553,0.0,-24.99,111
11647271,777320,990151510,2018-02-14,7005333,490558891.0,966887553,0.0,0.00,111
11647275,777320,990151510,2018-02-14,7046930,490558891.0,966887553,0.0,0.00,111
11648080,777320,990152430,2018-02-17,7004584,490558891.0,966890158,0.0,-24.99,111
11648082,777320,990152430,2018-02-17,7005333,490558891.0,966890158,0.0,0.00,111


##### What are some of these articles with high duplicate counts?

In [82]:
print("What is ARTICLE_ID 7001674?", products.query("ARTICLE_ID == 7001674")["SEGMENT_DESC"])
print("What is ARTICLE_ID 7046930?", products.query("ARTICLE_ID == 7046930")["SEGMENT_DESC"])
print("What is ARTICLE_ID 7074810?", products.query("ARTICLE_ID == 7074810")["SEGMENT_DESC"])
print("What is ARTICLE_ID 7009500?", products.query("ARTICLE_ID == 7009500")["SEGMENT_DESC"])
print("What is ARTICLE_ID 7004584?", products.query("ARTICLE_ID == 7004584")["SEGMENT_DESC"])

What is ARTICLE_ID 7001674? 43201    Lubrication Services
Name: SEGMENT_DESC, dtype: object
What is ARTICLE_ID 7046930? 44130    Inspection Services
Name: SEGMENT_DESC, dtype: object
What is ARTICLE_ID 7074810? 43981    Alignments
Name: SEGMENT_DESC, dtype: object
What is ARTICLE_ID 7009500? 44048    Alignments
Name: SEGMENT_DESC, dtype: object
What is ARTICLE_ID 7004584? 43557    Alignments
Name: SEGMENT_DESC, dtype: object


In [59]:
# sanity check: filter dataframe by the values from a duplicated row 
duplicate_test.query("TRAN_ID == 990150440 & DATE == '2018-02-10' & ARTICLE_ID == 7009500 & INDIV_ID == 490558891.0 & VEHICLE_ID == 967184333")

,STORE_ID,TRAN_ID,DATE,ARTICLE_ID,INDIV_ID,VEHICLE_ID,UNITS,SALES
13221393,777320,990150440,2018-02-10,7009500,490558891.0,967184333,0.0,79.99
13221445,777320,990150440,2018-02-10,7009500,490558891.0,967184333,0.0,79.99
13221561,777320,990150440,2018-02-10,7009500,490558891.0,967184333,0.0,79.99
13221608,777320,990150440,2018-02-10,7009500,490558891.0,967184333,0.0,79.99
13221655,777320,990150440,2018-02-10,7009500,490558891.0,967184333,0.0,79.99
...,...,...,...,...,...,...,...,...
13226842,777320,990150440,2018-02-10,7009500,490558891.0,967184333,0.0,79.99
13226889,777320,990150440,2018-02-10,7009500,490558891.0,967184333,0.0,79.99
13226941,777320,990150440,2018-02-10,7009500,490558891.0,967184333,0.0,79.99
13227007,777320,990150440,2018-02-10,7009500,490558891.0,967184333,0.0,79.99


In [148]:
pd.merge(duplicates, products, on="ARTICLE_ID").query("CATEGORY_CODE == 26.0 | CATEGORY_CODE == 44.0").query("count > 10")

,STORE_ID,TRAN_ID,DATE,ARTICLE_ID,INDIV_ID,VEHICLE_ID,UNITS,SALES,count,units_duplicated,PROD_GROUP_CODE,PROD_GROUP_DESC,CATEGORY_CODE,CATEGORY_DESC,SEGMENT_CODE,SEGMENT_DESC,CLASS_CODE,CLASS_DESC,DISCOUNT_FLAG,CROSS_SECTION,ASPECT_RATIO,RIM_SIZE
784305,517585,993007880,2018-02-19,7099717,608296416.0,956305582,4.0,572.00,12,44.0,5.0,Tires,26.0,Passenger Tires,31.0,Other Passenger Tires,43.0,Other Passenger Tires,N,NONE,NONE,NONE
841496,15253,992233980,2018-02-05,3816,554140625.0,949679781,1.0,51.00,16,15.0,5.0,Tires,26.0,Passenger Tires,27.0,Touring,36.0,Car & Minivan All-Season,N,215,60,16
854205,783816,990006970,2018-02-18,3432,286075260.0,949065545,1.0,39.68,12,11.0,5.0,Tires,26.0,Passenger Tires,27.0,Touring,33.0,Mass Market Tires,N,225,60,16
912670,22365,992601220,2018-02-23,6252,606595975.0,940333414,1.0,40.00,24,23.0,5.0,Tires,26.0,Passenger Tires,27.0,Touring,36.0,Car & Minivan All-Season,N,195,65,15
912809,303600,991978740,2018-02-18,15199,388079829.0,952997514,1.0,12.22,16,15.0,5.0,Tires,26.0,Passenger Tires,29.0,Performance Tires,38.0,Performance All-Season,N,205,65,15
943188,18155,992326640,2018-02-18,6597,317697317.0,925611896,1.0,48.34,12,11.0,5.0,Tires,26.0,Passenger Tires,29.0,Performance Tires,39.0,Touring H/V/Z Tires,N,215,45,17
955147,244193,991715800,2018-02-23,15454,321870904.0,940242961,1.0,70.07,12,11.0,5.0,Tires,26.0,Passenger Tires,29.0,Performance Tires,38.0,Performance All-Season,N,215,45,17
970378,8028,992142630,2018-02-17,4020,580842339.0,962620818,1.0,51.03,12,11.0,5.0,Tires,26.0,Passenger Tires,27.0,Touring,36.0,Car & Minivan All-Season,N,215,65,16
971100,22535,991835450,2018-02-09,106310,304287722.0,884795879,1.0,136.49,12,11.0,5.0,Tires,26.0,Passenger Tires,29.0,Performance Tires,39.0,Touring H/V/Z Tires,N,225,50,17
971646,22365,992594240,2018-02-02,189752,606595975.0,937380730,1.0,30.00,12,11.0,5.0,Tires,44.0,Light Truck Tires,47.0,Commercial Tires,51.0,Highway Tires,N,225,75,16


In [154]:
pd.merge(duplicate_test, products, on="ARTICLE_ID").query("STORE_ID == 517585 & TRAN_ID == 993007880 & DATE == '2018-02-19' & ARTICLE_ID == 7099717")

,STORE_ID,TRAN_ID,DATE,ARTICLE_ID,INDIV_ID,VEHICLE_ID,UNITS,SALES,PROD_GROUP_CODE,PROD_GROUP_DESC,CATEGORY_CODE,CATEGORY_DESC,SEGMENT_CODE,SEGMENT_DESC,CLASS_CODE,CLASS_DESC,DISCOUNT_FLAG,CROSS_SECTION,ASPECT_RATIO,RIM_SIZE
12357327,517585,993007880,2018-02-19,7099717,264794513.0,956305582,4.0,572.0,5.0,Tires,26.0,Passenger Tires,31.0,Other Passenger Tires,43.0,Other Passenger Tires,N,NONE,NONE,NONE
12357328,517585,993007880,2018-02-19,7099717,608296416.0,956305582,4.0,572.0,5.0,Tires,26.0,Passenger Tires,31.0,Other Passenger Tires,43.0,Other Passenger Tires,N,NONE,NONE,NONE
12357329,517585,993007880,2018-02-19,7099717,608296416.0,956305582,4.0,572.0,5.0,Tires,26.0,Passenger Tires,31.0,Other Passenger Tires,43.0,Other Passenger Tires,N,NONE,NONE,NONE
12357330,517585,993007880,2018-02-19,7099717,608296416.0,956305582,4.0,572.0,5.0,Tires,26.0,Passenger Tires,31.0,Other Passenger Tires,43.0,Other Passenger Tires,N,NONE,NONE,NONE
12357331,517585,993007880,2018-02-19,7099717,608296416.0,956305582,4.0,572.0,5.0,Tires,26.0,Passenger Tires,31.0,Other Passenger Tires,43.0,Other Passenger Tires,N,NONE,NONE,NONE
12357332,517585,993007880,2018-02-19,7099717,608296416.0,956305582,4.0,572.0,5.0,Tires,26.0,Passenger Tires,31.0,Other Passenger Tires,43.0,Other Passenger Tires,N,NONE,NONE,NONE
12357333,517585,993007880,2018-02-19,7099717,608296416.0,956305582,4.0,572.0,5.0,Tires,26.0,Passenger Tires,31.0,Other Passenger Tires,43.0,Other Passenger Tires,N,NONE,NONE,NONE
12357334,517585,993007880,2018-02-19,7099717,608296416.0,956305582,4.0,572.0,5.0,Tires,26.0,Passenger Tires,31.0,Other Passenger Tires,43.0,Other Passenger Tires,N,NONE,NONE,NONE
12357335,517585,993007880,2018-02-19,7099717,608296416.0,956305582,4.0,572.0,5.0,Tires,26.0,Passenger Tires,31.0,Other Passenger Tires,43.0,Other Passenger Tires,N,NONE,NONE,NONE
12357336,517585,993007880,2018-02-19,7099717,608296416.0,956305582,4.0,572.0,5.0,Tires,26.0,Passenger Tires,31.0,Other Passenger Tires,43.0,Other Passenger Tires,N,NONE,NONE,NONE


##### How many rows were duplicated more than 100 times? 

In [132]:
# how many rows were duplicated more than 100 times? 
over_100 = duplicates.query("count >= 100")

print("No. rows duplicated more than 100 times: ", over_100.shape[0])

print("Percent of rows that are duplicated: ", duplicates["count"].sum()/duplicate_test.shape[0]*100)

duplicates["units_duplicated"] = (duplicates["UNITS"]*duplicates["count"])-duplicates["UNITS"]

print("Expected no. units duplicated in sales_20180228.csv:",
      duplicates["units_duplicated"].sum())

No. rows duplicated more than 100 times:  45
Percent of rows that are duplicated:  17.606395092580634
Expected no. units duplicated in sales_20180228.csv: 238067.0


#### Test case: April 2017 sales data

Duplicates test on second sales file...

In [45]:
%%time

duplicate_test_2 = pd.read_csv("sales_20170430.csv", sep="|", parse_dates=["DATE"],
                             usecols=["STORE_ID","TRAN_ID","ARTICLE_ID","INDIV_ID","VEHICLE_ID","DATE","UNITS","SALES"], 
                             dtype={"STORE_ID":np.int64,
                                    "TRAN_ID":np.int64,
                                    "ARTICLE_ID":np.int64,
                                    "VEHICLE_ID":int,
                                    "UNITS":np.float64,
                                    "SALES":np.float64},
                             low_memory=False)
print("Parsing complete. Shape of dataframe: ", duplicate_test_2.shape)

Parsing complete. Shape of dataframe:  (14912797, 8)
CPU times: user 4.63 s, sys: 1.23 s, total: 5.86 s
Wall time: 6.06 s


In [46]:
# Group by all columns and count the number of duplicates
duplicates_2 = duplicate_test_2.groupby(duplicate_test_2.columns.tolist()).size().reset_index().rename(columns={0:'count'})

# Filter the dataframe to keep only rows that have more than one occurrence
duplicates_2 = duplicates_2[duplicates_2['count'] > 1]

In [61]:
# sanity check: filter dataframe by the values from a duplicated row 
duplicate_test_2.query("TRAN_ID == 990047830 & DATE == '2017-04-20' & ARTICLE_ID == 7009500 & INDIV_ID == 490558891.0 & VEHICLE_ID == 957633518")

,STORE_ID,TRAN_ID,DATE,ARTICLE_ID,INDIV_ID,VEHICLE_ID,UNITS,SALES
12370299,777320,990047830,2017-04-20,7009500,490558891.0,957633518,0.0,79.99
12370339,777320,990047830,2017-04-20,7009500,490558891.0,957633518,0.0,79.99
12370461,777320,990047830,2017-04-20,7009500,490558891.0,957633518,0.0,79.99
12370501,777320,990047830,2017-04-20,7009500,490558891.0,957633518,0.0,79.99
12370541,777320,990047830,2017-04-20,7009500,490558891.0,957633518,0.0,79.99
...,...,...,...,...,...,...,...,...
12374858,777320,990047830,2017-04-20,7009500,490558891.0,957633518,0.0,79.99
12374898,777320,990047830,2017-04-20,7009500,490558891.0,957633518,0.0,79.99
12374938,777320,990047830,2017-04-20,7009500,490558891.0,957633518,0.0,79.99
12375004,777320,990047830,2017-04-20,7009500,490558891.0,957633518,0.0,79.99


##### How many rows were duplicated more than 100 times? 

In [133]:
# how many rows were duplicated more than 100 times? 
over_100 = duplicates_2.query("count >= 100")

print("No. rows duplicated more than 100 times: ", over_100.shape[0])

print("Percent of rows that are duplicated: ", duplicates_2["count"].sum()/duplicate_test_2.shape[0]*100)

duplicates_2["units_duplicated"] = (duplicates_2["UNITS"]*duplicates_2["count"])-duplicates_2["UNITS"]

print("Expected no. units duplicated in sales_20170430.csv:",
      duplicates_2["units_duplicated"].sum())

No. rows duplicated more than 100 times:  40
Percent of rows that are duplicated:  18.601654672828978
Expected no. units duplicated in sales_20170430.csv: 245878.0


### Test case: January 2017

In [113]:
%%time

duplicate_test_3 = pd.read_csv("sales_20170131.csv", sep="|", parse_dates=["DATE"],
                             usecols=["STORE_ID","TRAN_ID","ARTICLE_ID","INDIV_ID","VEHICLE_ID","DATE","UNITS","SALES"], 
                             dtype={"STORE_ID":np.int64,
                                    "TRAN_ID":np.int64,
                                    "ARTICLE_ID":np.int64,
                                    #"VEHICLE_ID":int,
                                    "UNITS":np.float64,
                                    "SALES":np.float64},
                             low_memory=False)
print("Parsing complete. Shape of dataframe: ", duplicate_test_3.shape)

Parsing complete. Shape of dataframe:  (13251659, 8)
CPU times: user 4.08 s, sys: 900 ms, total: 4.98 s
Wall time: 5.01 s


In [135]:
# Group by all columns and count the number of duplicates
duplicates_3 = duplicate_test_3.groupby(duplicate_test_3.columns.tolist()).size().reset_index().rename(columns={0:'count'})

# Filter the dataframe to keep only rows that have more than one occurrence
duplicates_3 = duplicates_3[duplicates_3['count'] > 1]

print("Percent of rows that are duplicated: ", duplicates_3["count"].sum()/duplicate_test_3.shape[0]*100)

duplicates_3["units_duplicated"] = (duplicates_3["UNITS"]*duplicates_3["count"])-duplicates_3["UNITS"]

print("Expected no. units duplicated in sales_20170131.csv:",
      duplicates_3["units_duplicated"].sum())

Percent of rows that are duplicated:  14.365748469682174
Expected no. units duplicated in sales_20170131.csv: 92810.0


### Test case: November 2017

In [119]:
%%time

duplicate_test_4 = pd.read_csv("sales_20171130.csv", sep="|", parse_dates=["DATE"],
                             usecols=["STORE_ID","TRAN_ID","ARTICLE_ID","INDIV_ID","VEHICLE_ID","DATE","UNITS","SALES"], 
                             dtype={"STORE_ID":np.int64,
                                    "TRAN_ID":np.int64,
                                    "ARTICLE_ID":np.int64,
                                    #"VEHICLE_ID":int,
                                    "UNITS":np.float64,
                                    "SALES":np.float64},
                             low_memory=False)
print("Parsing complete. Shape of dataframe: ", duplicate_test_4.shape)

Parsing complete. Shape of dataframe:  (1648017, 8)
CPU times: user 536 ms, sys: 145 ms, total: 681 ms
Wall time: 695 ms


In [136]:
# Group by all columns and count the number of duplicates
duplicates_4 = duplicate_test_4.groupby(duplicate_test_4.columns.tolist()).size().reset_index().rename(columns={0:'count'})

# Filter the dataframe to keep only rows that have more than one occurrence
duplicates_4 = duplicates_4[duplicates_4['count'] > 1]

print("Percent of rows that are duplicated: ", duplicates_4["count"].sum()/duplicate_test_4.shape[0]*100)

duplicates_4["units_duplicated"] = (duplicates_4["UNITS"]*duplicates_4["count"])-duplicates_4["UNITS"]

print("Expected no. units duplicated in sales_20171130.csv:",
      duplicates_4["units_duplicated"].sum())

Percent of rows that are duplicated:  12.8308142452414
Expected no. units duplicated in sales_20171130.csv: 6380.0


## Summary of findings

- Columns validated

- Dates validated; found one missing date (2017-11-23)

- Duplicate rows identified - some duplicated hundreds of times. Seems to be an issue with store # 777320.

- 353 zip codes are not in tire_sales -- how could this many be excluded? Seems to span across all states, stores, and dates.

- These articles are missing from July-Aug 2017 sales data: ['3419','3428','3430','3432','3434','3438','3439','3442','3444','3447','3449','3429', '3431']

# Data Validation: master_sample.csv

## Identify NA values

The following code chunks identify the number of rows in each column containing NA values. Some columns we can probably ignore (MSB_INDIV_ID, EMAIL_OPTIN_IND, etc.). 

The columns where we would not expect NA values are: INDIV_ID, MAKE, MODEL, and MODEL_YEAR.  

In [ ]:
# init a mask that will id NA values as True
na_mask = df.isna()

na_count = na_mask.sum()

print(na_count)

In [ ]:
df_drop = df.drop(columns=['MZB_INDIV_ID', 'EMAIL_OPTIN_IND', 'AH1_RES_BUS_INDC', 'SUPP1_BUS_PANDER'])
df_drop.head()

In [ ]:
# create df of rows where INDIV_ID = NaN
na_indiv = df_drop[df_drop['INDIV_ID'].isna()]
na_indiv

In [ ]:
# use pandas-profiling to generate a report for the dataframe
report = pandas_profiling.ProfileReport(na_indiv)

# save the report to an HTML file
report.to_file('validation_reports/na_indiv_report.html')

In [ ]:
# create na reports for these columns as well: MAKE, MODEL, SUB-MODEL, MODEL_YEAR

na_cols = ['MAKE', 'MODEL', 'SUB_MODEL', 'MODEL_YEAR']
na_df = []
directory = 'validation_reports/'

for col in na_cols:
    df = df_drop[df_drop[col].isna()]
    report = pandas_profiling.ProfileReport(df)
    report.to_file(os.path.join(directory, col + '.html'))


In [ ]:
# get pandas profile on entire master_sample dataframe (excluding dropped columns)
report = pandas_profiling.ProfileReport(df_drop)
report.to_file('validation_reports/master_sample.html')

In [ ]:
test = df_drop.loc[df_drop['ARTICLE_ID'] == 15148]

len(test['PROD_GROUP_CODE'].unique())

In [ ]:
# check that unique ARTICLE_IDs retain the same code values
# throughout the df

unique_articles = df_drop['ARTICLE_ID'].unique()

cols_to_check = ['PROD_GROUP_CODE', 'CATEGORY_CODE', 'SEGMENT_CODE', 'CLASS_CODE', 'CROSS_SECTION', 'ASPECT_RATIO','RIM_SIZE']

for value in unique_articles:
    temp = df_drop[df_drop['ARTICLE_ID'] == value]
    for col in cols_to_check:
        if len(temp[col].unique()) == 1:
            print("No change in value.")
            continue
        else:
            print("Value changed for ARTICLE_ID: ", value, " in: ", col)

In [ ]:
grouped_class_desc = df_drop.groupby('CLASS_DESC').count()
grouped_class_desc

In [ ]:
grouped_stores = df_drop.groupby('STORE_ID').count()
len(df_drop['ZIP_CODE'].unique())

In [ ]:
snow_tire_codes = [1136, 76302, 21745]

snow_tires_df = df_drop[df_drop['CLASS_CODE'].isin(snow_tire_codes)]
snow_tires_df.groupby('STATE_CODE').count().sort_values('TRAN_ID', ascending=False)

In [ ]:
test = df_drop[df_drop['STATE_CODE'] == 'CO'] # filters df for CO 

test_new = test.groupby('CLASS_DESC').count().sort_values('UNITS', ascending=False) # groups CO df by class_desc, sorts by units

#temp_list = test_new['UNITS'].nlargest(5).index.tolist() # gets the index vals of top 5 rows and stores as list
test_units = test_new['UNITS'].nlargest(5).tolist()
test_units

#dict.update({state: temp_list})

In [ ]:
# get list of state codes from the df
states = df_drop['STATE_CODE'].unique()
states

top_5_tires_per_state = {}

for state in states:
    # filter df by state code
    temp = df_drop[df_drop['STATE_CODE'] == state].groupby('CLASS_DESC').count().sort_values('UNITS', ascending=False)

    # gets the index vals of top 5 rows and stores as list
    class_list = temp['UNITS'].nlargest(5).index.tolist()
    units_list = temp['UNITS'].nlargest(5).tolist()

    # adds key-value to dict
    top_5_tires_per_state.update({state: [class_list, units_list]})


In [ ]:
top_5_tires_per_state

In [ ]:
least_5_tires_per_state = {}

for state in states:
    # filter df by state code
    temp = df_drop[df_drop['STATE_CODE'] == state].groupby('CLASS_DESC').count().sort_values('UNITS', ascending=True)

    # gets the index vals of top 5 rows and stores as list
    class_list = temp['UNITS'].nsmallest(5).index.tolist()
    units_list = temp['UNITS'].nsmallest(5).tolist()

    # adds key-value to dict
    least_5_tires_per_state.update({state: [class_list, units_list]})


In [ ]:
least_5_tires_per_state